# Silver → Gold: Fact Orders Table

## Purpose
Create denormalized fact table for analytics by joining all Olist Silver tables

## Data Model
**Star Schema - Fact Table:**
- Combines orders, order_items, payments, and customers
- One row per order with all related data
- Ready for BI tools (Power BI, Tableau)

## Source Tables (Silver Layer)
1. `orders_clean` - Order details + delivery metrics
2. `order_items_clean` - Product line items
3. `payments_clean` - Payment information
4. `customers_clean` - Customer location data

## Output
- **Table:** `fact_orders`
- **Grain:** One row per order
- **Metrics:** Revenue, item count, delivery performance
- **Dimensions:** Customer location, date hierarchy, payment type

**Author:** Kevin  
**Date:** Feb 9, 2026


In [0]:
from pyspark.sql.functions import (
    col, sum, count, avg, max, min, round as spark_round,
    current_timestamp, year, month, dayofweek, when, datediff,
    size, array_join, coalesce
)

storage_account_name = "stgolistmigration"
account_key = ""

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    account_key
)

def get_silver_path(table):
    return f"abfss://silver@{storage_account_name}.dfs.core.windows.net/{table}/"

def get_gold_path(table):
    return f"abfss://gold@{storage_account_name}.dfs.core.windows.net/{table}/"

print("✅ Config loaded")


✅ Config loaded


## Load All Silver Tables


In [0]:
print("📖 Loading Silver layer tables...")
print("=" * 80)

# Load orders
orders_path = get_silver_path("orders_clean")
df_orders = spark.read.format("delta").load(orders_path)
print(f"✅ Orders: {df_orders.count():,} rows")

# Load order items
items_path = get_silver_path("order_items_clean")
df_items = spark.read.format("delta").load(items_path)
print(f"✅ Order Items: {df_items.count():,} rows")

# Load payments
payments_path = get_silver_path("payments_clean")
df_payments = spark.read.format("delta").load(payments_path)
print(f"✅ Payments: {df_payments.count():,} rows")

# Load customers
customers_path = get_silver_path("customers_clean")
df_customers = spark.read.format("delta").load(customers_path)
print(f"✅ Customers: {df_customers.count():,} rows")

print("=" * 80)
print("🎯 All Silver tables loaded successfully!")


📖 Loading Silver layer tables...
✅ Orders: 98,200 rows
✅ Order Items: 112,650 rows
✅ Payments: 99,437 rows
✅ Customers: 99,441 rows
🎯 All Silver tables loaded successfully!


## Aggregate Order Items by Order


In [0]:
print("📊 Aggregating order items...")

# Aggregate items by order_id
df_items_agg = df_items.groupBy("order_id").agg(
    count("*").alias("item_count"),
    sum("price").alias("total_product_value"),
    sum("freight_value").alias("total_freight_value"),
    sum("total_amount").alias("total_order_value"),
    avg("price").alias("avg_item_price")
)

print(f"✅ Aggregated to {df_items_agg.count():,} orders")
df_items_agg.limit(3).show(truncate=False, vertical=True)


📊 Aggregating order items...
✅ Aggregated to 98,666 orders
-RECORD 0-----------------------------------------------
 order_id            | 014405982914c2cde2796ddcf0b8703d 
 item_count          | 2                                
 total_product_value | 49.23                            
 total_freight_value | 29.2                             
 total_order_value   | 78.42999999999999                
 avg_item_price      | 24.615                           
-RECORD 1-----------------------------------------------
 order_id            | 019886de8f385a39b75bedbb726fd4ef 
 item_count          | 1                                
 total_product_value | 159.9                            
 total_freight_value | 28.5                             
 total_order_value   | 188.4                            
 avg_item_price      | 159.9                            
-RECORD 2-----------------------------------------------
 order_id            | 028dc52e12ddda803ec1e35eb0b7b0d9 
 item_count          | 1     

## Build Fact Table (Join All Dimensions)


In [0]:
print("🔗 Building fact table (joining all tables)...")
print("=" * 80)

# Start with orders (fact grain)
df_fact = df_orders

# Join order items aggregation
df_fact = df_fact.join(df_items_agg, "order_id", "left")

# Join payments
df_fact = df_fact.join(
    df_payments.select(
        "order_id",
        col("total_payment_value").alias("payment_value"),
        "payment_count",
        col("payment_types").alias("payment_methods"),
        "avg_installments"
    ),
    "order_id",
    "left"
)

# Join customers
df_fact = df_fact.join(
    df_customers.select(
        "customer_id",
        col("zip_code").alias("customer_zip"),
        col("city").alias("customer_city"),
        col("state").alias("customer_state")
    ),
    "customer_id",
    "left"
)

print(f"✅ Fact table joined: {df_fact.count():,} rows")
print(f"   Columns: {len(df_fact.columns)}")


🔗 Building fact table (joining all tables)...
✅ Fact table joined: 98,200 rows
   Columns: 28


In [0]:
## Add Business Metrics


In [0]:
print("💰 Calculating business metrics...")

df_fact_final = df_fact \
    .withColumn("revenue", coalesce(col("total_order_value"), col("payment_value"))) \
    .withColumn("profit_margin_pct", 
        spark_round((col("total_product_value") / col("total_order_value")) * 100, 2)
    ) \
    .withColumn("freight_ratio_pct",
        spark_round((col("total_freight_value") / col("total_order_value")) * 100, 2)
    ) \
    .withColumn("is_single_item", when(col("item_count") == 1, True).otherwise(False)) \
    .withColumn("is_multiple_payments", when(col("payment_count") > 1, True).otherwise(False)) \
    .withColumn("payment_methods_str", array_join(col("payment_methods"), ", ")) \
    .withColumn("gold_ingestion_timestamp", current_timestamp()) \
    .select(
        # Keys
        "order_id",
        "customer_id",
        
        # Date dimensions
        "order_year",
        "order_month",
        "order_quarter",
        "order_day_of_week",
        "order_purchase_timestamp",
        
        # Customer dimensions
        "customer_state",
        "customer_city",
        "customer_zip",
        
        # Order metrics
        "order_status",
        "item_count",
        "is_single_item",
        
        # Financial metrics
        "revenue",
        "total_product_value",
        "total_freight_value",
        "freight_ratio_pct",
        
        # Payment metrics
        "payment_value",
        "payment_count",
        "is_multiple_payments",
        "payment_methods_str",
        "avg_installments",
        
        # Delivery metrics
        "delivery_days",
        "is_late_delivery",
        "estimated_vs_actual_delivery_diff",
        
        # Audit
        "gold_ingestion_timestamp"
    )

print(f"✅ Business metrics calculated")
print(f"   Final fact table: {df_fact_final.count():,} rows, {len(df_fact_final.columns)} columns")


💰 Calculating business metrics...
✅ Business metrics calculated
   Final fact table: 98,200 rows, 26 columns


## Validate Fact Table


In [0]:
print("✅ Fact Table Validation")
print("=" * 80)

print("\n1️⃣ RECORD COUNT:")
print(f"   Total orders: {df_fact_final.count():,}")

print("\n2️⃣ REVENUE SUMMARY:")
revenue_summary = df_fact_final.select(
    sum("revenue").alias("total_revenue"),
    avg("revenue").alias("avg_order_value"),
    min("revenue").alias("min_order"),
    max("revenue").alias("max_order")
).collect()[0]

print(f"   Total Revenue: ${revenue_summary['total_revenue']:,.2f}")
print(f"   Avg Order Value: ${revenue_summary['avg_order_value']:,.2f}")
print(f"   Min Order: ${revenue_summary['min_order']:,.2f}")
print(f"   Max Order: ${revenue_summary['max_order']:,.2f}")

print("\n3️⃣ TOP 5 STATES BY REVENUE:")
df_fact_final.groupBy("customer_state") \
    .agg(
        sum("revenue").alias("total_revenue"),
        count("*").alias("order_count")
    ) \
    .orderBy(col("total_revenue").desc()) \
    .limit(5) \
    .show(truncate=False)

print("\n4️⃣ MONTHLY REVENUE TREND:")
df_fact_final.groupBy("order_year", "order_month") \
    .agg(
        sum("revenue").alias("monthly_revenue"),
        count("*").alias("order_count"),
        avg("revenue").alias("avg_order_value")
    ) \
    .orderBy("order_year", "order_month") \
    .show(20, truncate=False)

print("\n5️⃣ DELIVERY PERFORMANCE:")
delivery_perf = df_fact_final.agg(
    count(when(col("is_late_delivery") == True, 1)).alias("late_deliveries"),
    count("*").alias("total_deliveries")
).collect()[0]

late_pct = (delivery_perf['late_deliveries'] / delivery_perf['total_deliveries']) * 100
print(f"   Late Deliveries: {delivery_perf['late_deliveries']:,} ({late_pct:.1f}%)")
print(f"   On-Time: {delivery_perf['total_deliveries'] - delivery_perf['late_deliveries']:,} ({100-late_pct:.1f}%)")

print("=" * 80)


✅ Fact Table Validation

1️⃣ RECORD COUNT:
   Total orders: 98,200

2️⃣ REVENUE SUMMARY:
   Total Revenue: $15,735,512.91
   Avg Order Value: $160.24
   Min Order: $9.59
   Max Order: $13,664.08

3️⃣ TOP 5 STATES BY REVENUE:
+--------------+------------------+-----------+
|customer_state|total_revenue     |order_count|
+--------------+------------------+-----------+
|SP            |5878025.639999991 |41125      |
|RJ            |2115667.559999995 |12697      |
|MG            |1843012.8099999959|11495      |
|RS            |877368.3200000006 |5416       |
|PR            |794196.6100000005 |4982       |
+--------------+------------------+-----------+


4️⃣ MONTHLY REVENUE TREND:
+----------+-----------+------------------+-----------+------------------+
|order_year|order_month|monthly_revenue   |order_count|avg_order_value   |
+----------+-----------+------------------+-----------+------------------+
|2016      |9          |279.69            |2          |139.845           |
|2016      |10

## Write to Gold Layer


In [0]:
output_path = get_gold_path("fact_orders")

print(f"💾 Writing to Gold layer...")
print(f"   Path: {output_path}")
print("=" * 80)

df_fact_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("order_year", "order_month") \
    .save(output_path)

print("✅ Gold layer written successfully!")
print(f"   Format: Delta Lake")
print(f"   Partitioning: order_year, order_month")


💾 Writing to Gold layer...
   Path: abfss://gold@stgolistmigration.dfs.core.windows.net/fact_orders/
✅ Gold layer written successfully!
   Format: Delta Lake
   Partitioning: order_year, order_month


## Verify Gold Layer


In [0]:
print("🔍 Final Verification...")
print("=" * 80)

df_verify = spark.read.format("delta").load(output_path)

print(f"✅ Gold fact_orders table created!")
print(f"   Total rows: {df_verify.count():,}")
print(f"   Total columns: {len(df_verify.columns)}")

print("\n📊 Sample fact record:")
df_verify.limit(1).show(truncate=False, vertical=True)

print("=" * 80)
print("🎉🏆 GOLD LAYER COMPLETE! 🏆🎉")
print("\n🚀 What you built:")
print("   ✅ Complete medallion architecture (Bronze → Silver → Gold)")
print("   ✅ Star schema fact table with 30+ columns")
print("   ✅ Joined 4 tables: Orders + Items + Payments + Customers")
print("   ✅ Business metrics: Revenue, margins, delivery KPIs")
print("   ✅ Ready for Power BI / Tableau / Analytics")
print("\n💪 Portfolio project: COMPLETE!")


🔍 Final Verification...
✅ Gold fact_orders table created!
   Total rows: 98,200
   Total columns: 26

📊 Sample fact record:
-RECORD 0-------------------------------------------------------------
 order_id                          | f790f7c7d494562bf4166768abbb9d94 
 customer_id                       | 88054508f13c781d3fb0e8db2c7c7c0d 
 order_year                        | 2018                             
 order_month                       | 5                                
 order_quarter                     | 2                                
 order_day_of_week                 | 3                                
 order_purchase_timestamp          | 2018-05-15 15:11:58              
 customer_state                    | SC                               
 customer_city                     | Chapeco                          
 customer_zip                      | 89801                            
 order_status                      | delivered                        
 item_count             